In [12]:
import config, sfa, poor_security

In [13]:
from importlib import reload

### Load a Database

In [84]:
reload(sfa)
reload(config)
# initiate SnowflakeHelper with test type and dataset scale specified
x = sfa.SFTPC(warehouse="TEST9000", 
                test="ds",
                scale=1,
                n="01",
                verbose_query=False, verbose=True)

x.connect()

x.role("SYSADMIN")

r1 = x.show_warehouses()

x.warehouse_create()

x.database_drop()
x.gcs_integration_drop()

x.database_create()
x.database_setup_role("SYSADMIN")
x.database_use()
x.create_schema(schema_file=config.fp_schema + config.sep + "ansi_ds.sql")
x.create_named_file_format()
x.gcs_integration_create()
x.grant_storage_integration_access()
x.create_stage()

x.gcs_inventory()
x.import_data()

i2 = x.gcs_integration_list()
r2 = x.show_warehouses()

r3 = x.query_n(n=1)

sfc.close()

Using configuration: user:colin
Password: jfguye229ekkeQA
Account: wja13212
Connector cursor created.
Loading table: call_center
Done!

Loading table: catalog_page
Done!

Loading table: catalog_returns
Done!

Loading table: catalog_sales
Done!

Loading table: customer
Done!

Loading table: customer_address
Done!

Loading table: customer_demographics
Done!

Loading table: date_dim
Done!

Loading table: household_demographics
Done!

Loading table: income_band
Done!

Loading table: inventory
Done!

Loading table: item
Done!

Loading table: promotion
Done!

Loading table: reason
Done!

Loading table: ship_mode
Done!

Loading table: store
Done!

Loading table: store_returns
Done!

Loading table: store_sales
Done!

Loading table: time_dim
Done!

Loading table: warehouse
Done!

Loading table: web_page
Done!

Loading table: web_returns
Done!

Loading table: web_sales
Done!

Loading table: web_site
Done!



### Query a Database

In [112]:
reload(sfa)
reload(config)
# initiate SnowflakeHelper with test type and dataset scale specified
x = sfa.SFTPC(warehouse="TEST9000", 
                test="ds",
                scale=1,
                n="01",
                verbose_query=False, verbose=True)

x.connect()
x.role("SYSADMIN")
x.warehouse_use()
x.database_use()

r = x.query_n(n=1, verbose=True, verbose_out=True)
df, qid = x.query_history()

sfc.close()

Using configuration: user:colin
Password: jfguye229ekkeQA
Account: wja13212
Connector cursor created.
TPC-DS dsqgen parameters
command & kwargs: ['./dsqgen', '-VERBOSE', '-SCALE', '1', '-RNGSEED', '13', '-TEMPLATE', 'query1.tpl', '-FILTER', '-DIALECT', 'sqlserver_tpc', '-DIRECTORY', '/home/colin/code/bq_snowflake_benchmark/tpl/sf_ds']
cwd: /home/colin/code/bq_snowflake_benchmark/ds/v2.11.0rc2/tools

QUERY: 1

Standard Out:

with customer_total_return as
(select sr_customer_sk as ctr_customer_sk
,sr_store_sk as ctr_store_sk
,sum(SR_RETURN_AMT) as ctr_total_return
from store_returns
,date_dim
where sr_returned_date_sk = d_date_sk
and d_year =1998
group by sr_customer_sk
,sr_store_sk)
 select  c_customer_id
from customer_total_return ctr1
,store
,customer
where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
from customer_total_return ctr2
where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
and s_store_sk = ctr1.ctr_store_sk
and s_state = 'TN'
and ctr1.ctr_customer_sk = c_customer_sk
o

In [113]:
df

,QUERY_ID,QUERY_TEXT,DATABASE_NAME,SCHEMA_NAME,QUERY_TYPE,SESSION_ID,USER_NAME,ROLE_NAME,WAREHOUSE_NAME,WAREHOUSE_SIZE,...,TRANSACTION_BLOCKED_TIME,OUTBOUND_DATA_TRANSFER_CLOUD,OUTBOUND_DATA_TRANSFER_REGION,OUTBOUND_DATA_TRANSFER_BYTES,INBOUND_DATA_TRANSFER_CLOUD,INBOUND_DATA_TRANSFER_REGION,INBOUND_DATA_TRANSFER_BYTES,CREDITS_USED_CLOUD_SERVICES,LIST_EXTERNAL_FILE_TIME,RELEASE_VERSION
0,01941e92-022c-33ff-0009-c08300023b3e,create table web_sales\n(\n ws_sold_date_sk...,DS_1GB_01,PUBLIC,CREATE_TABLE,41884553701,COLIN,SYSADMIN,TEST9000,None,...,0,None,None,0,None,None,0,0.000021000,0,4.14.4
1,01941e92-020e-1945-0009-c08300023b42,create table catalog_sales\n(\n cs_sold_dat...,DS_1GB_01,PUBLIC,CREATE_TABLE,41884553701,COLIN,SYSADMIN,TEST9000,None,...,0,None,None,0,None,None,0,0.000020000,0,4.14.4
2,01941e92-0251-1fd8-0009-c083000248ee,create table store_sales\n(\n ss_sold_date_...,DS_1GB_01,PUBLIC,CREATE_TABLE,41884553701,COLIN,SYSADMIN,TEST9000,None,...,0,None,None,0,None,None,0,0.000023000,0,4.14.4
3,01941e92-0293-b6f3-0009-c08300023b46,create or replace file format csv_file_format\...,DS_1GB_01,PUBLIC,CREATE,41884553701,COLIN,SYSADMIN,TEST9000,None,...,0,None,None,0,None,None,0,0.000010000,0,4.14.4
4,01941e92-0249-9cd1-0009-c083000248f2,CREATE STORAGE INTEGRATION ds_1GB_01_gcs_integ...,DS_1GB_01,PUBLIC,CREATE,41884553701,COLIN,SYSADMIN,TEST9000,None,...,0,None,None,0,None,None,0,0.000014000,0,4.14.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,01941f49-02ff-1490-0009-c08300023cbe,USE ROLE SYSADMIN,None,None,USE,41884545481,COLIN,SYSADMIN,None,None,...,0,None,None,0,None,None,0,0.000011000,0,4.14.4
96,01941f49-02a5-b7c4-0009-c08300024a82,USE WAREHOUSE TEST9000,None,None,USE,41884545481,COLIN,SYSADMIN,None,None,...,0,None,None,0,None,None,0,0.000008000,0,4.14.4
97,01941f49-022e-889e-0009-c08300023cc2,USE DATABASE ds_1GB_01,None,None,USE,41884545481,COLIN,SYSADMIN,TEST9000,None,...,0,None,None,0,None,None,0,0.000015000,0,4.14.4
98,01941f49-0243-157a-0009-c08300024a86,with customer_total_return as\n(select sr_cust...,DS_1GB_01,PUBLIC,SELECT,41884545481,COLIN,SYSADMIN,TEST9000,None,...,0,None,None,0,None,None,0,0.000021000,0,4.14.4
